# Import Package

In [1]:
import sys
 
# appending a path
sys.path.append('')
 
import os
import pandas as pd
import numpy as np
import pickle
# import SFR_PD_Recalibration_2023_Lib as tool
# import Concentra_SFR_Fit_for_Use_2024_Lib as lib
from matplotlib import pyplot as plt
import math

import Lib_RWA as Lib_RWA

del sys.modules['Lib_RWA']  # Remove the module from sys.modules to force a reload

import Lib_RWA as Lib_RWA

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\georgez\AppData\Local\Temp\ipykernel_24240\3764690850.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Inputs

In [4]:
# snapshot_num = 202212 # 20250412 George commented out
# snapshot = str(snapshot_num) # 20250412 George commented out
# snapshot_date = '2022-12-31' # 20250412 George commented out

snapshot_num = 202412 # 20250412 George added
snapshot = str(snapshot_num) # 20250412 George added
snapshot_date = '2024-12-31' # 20250412 George added


#CAR 2023 Ch5 prescribed
correlation_residential_mortgages = 0.15
correlation_residential_mortgages_rental = 0.22

CMHC_pd = 0.0001
CMHC_lgd = 0.11 #according to the newly developed Sovereign LGD model

lgd_gen_floor = 0.1 #general floor by CAR 2023


# Set Folders

In [5]:
# Get current working directory

current_dir = os.getcwd()  # 20250403 George added

# code_dir= "C:\\Users" +'\\' + username + '\\' + "Equitable Bank\\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\\RWA\\code" # 20250403 George commented out Francesca/Joseph's code

code_dir = current_dir # 20250403 George added

if int(snapshot_num) == 202412:
    input_dir = code_dir + '\\..\\' + "Inputs" # 20250406 George added
    Output_dir = code_dir + '\\..\\' + "Outputs" #20250406 George added

if int(snapshot_num) == 202212:
    input_dir = code_dir + '\\..\\' + "Dec. 2022 RWA Inputs" #20250410 George added
    Output_dir = code_dir + '\\..\\' + "Replicated Dec 2022 Outputs" #20250410 George added

# Load Corporate Loan Data

In [6]:
# Load Corporate_Loans_Breakdown.xlsx, sheet = Corp_Loans

Corp_Loans = pd.read_excel(Output_dir + '\\' + 'Corporate_Loans_Breakdown.xlsx', sheet_name='Corp_Loans')

In [14]:
# describe Remainging_Term

Corp_Loans['Remaining_Term'].describe()

KeyError: 'Remaining_Term'

In [ ]:
# Set rwa_data as a copy
rwa_data = Corp_Loans.copy()



# Add Column Insured_class

In [ ]:
# if Corporate_Applicant_Ind has only one value 1, then add a column Insured_class with value "Uninsured"

if rwa_data['Corporate_Applicant_Ind'].nunique() == 1 and rwa_data['Corporate_Applicant_Ind'].iloc[0] == 1:
    rwa_data['Insured_class'] = 'Uninsured'

# Add Column corr_uninsured

In [ ]:
# rwa_data['corr_uninsured'] =rwa_data['Occupancy_Status'].map(lambda x:0.22 if x in ['Rented','Unknown'] else 0.15) # 20250412 George commented out
rwa_data['corr_uninsured'] =rwa_data['Occupancy'].map(lambda x:0.15 if x in ['owner occupied','Owner Occupied','Owner_Occupied','owner_occupied'] else 0.22) # 20250412 George added

In [61]:
rwa_data.columns.values

array(['Mortgage_Number', 'Snapshot_Date', 'PD_Pre_MOC',
       'PD_Post_MOC_Pre_Adj', 'PD_Post_MOC', 'MRS_Bin_PD', 'Insured_Ind',
       'Appraisal_Value_v2', 'Combo_LTV_Insured_Ind',
       'Combo_Province_Metro_Override_WOE', 'def_90dpd_utp_filldown',
       'Concentra_Legacy_Ind', 'Occupancy_Status',
       'Original_Loan_to_Value', 'FSA', 'Appraisal_Date_v2',
       'Current_Principal_Balance_CB', 'Insurer', 'Original_Loan_Amount',
       'Original_Loan_to_Value', 'MRS_Bin_LGD', 'Pred_LGD', 'Mapped_LGD',
       'Model_LGD', 'Remaining_Term', 'Insured_class', 'corr_uninsured'],
      dtype=object)

# Add Other columns 

In [ ]:
# dt_adjust={1:0.016497,2:0.040137,3:0.104959,4:0.180642}   #20250412 George commented out the numbers are not the latest and this will not be used in the final result
# rwa_data['LGD_DT_Adjusted']=rwa_data['MRS_Bin_LGD'].map(dt_adjust) #20250412 George commented out the numbers are not the latest and this will not be used in the final result

# if LGD_DT_Adjusted is not in the columns, add it as NaN

if 'LGD_DT_Adjusted' not in rwa_data.columns: # 20250412 George added
    rwa_data['LGD_DT_Adjusted'] = np.nan  # 20250412 George added, the numbers are not the latest and this will not be used in the final result

rwa_data['Advance_Amount'] = rwa_data.loc[:,'AdvanceAmout_EQB_Exp']
rwa_data['Years_to_maturity'] = rwa_data.loc[:,'Remaining_Term']/12
rwa_data['EAD'] = rwa_data.loc[:,'RemainingPrincipal_Excl_Partner']

# RWA with 0 Addon

In [63]:
# Force Addon to be 0

rwa_raw_data_5 = rwa_data.copy()
rwa_raw_data_5['AddOn']=0


In [ ]:
# Run RWA calculation function

[df_out, rwa_by_Insured_class, rwa_by_MRS_Bin, res_data]  = Lib_RWA.rwa_calculation(rwa_raw_data_5, lgd_gen_floor, CMHC_lgd, CMHC_pd)



0
1
2


In [ ]:
# Save Data
     
with pd.ExcelWriter(
   Output_dir + '\concentra_rwa_addon_zero.xlsx') as writer: #20250406 George added
          
     res_data.to_excel(writer, sheet_name="RWA_Data")

# Borrow Add on Impact from EQB

In [67]:
# We need to add a cross table of lgd_bins and PD bins for rwa with add-on. However, we don't have account level addon info. therefore we take the impact from EQB at the Combo_LTV_Insured_Ind X Combo_Province_Metro_Override_WOE level

# addon_impact =pd.read_excel(r"C:\Users\francescav\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Outputs\addon_impact.xlsx")  # 20250406 George commented out Francesca's code

addon_impact =pd.read_excel(Output_dir+r"\addon_impact.xlsx") # 20250406 George added

rwa_raw_data = res_data.copy() # 20250411 George added because a new res_data is created in the above function

In [68]:
# 20250403 - Francesca Add on already has mapped values !
addon_impact.Combo_Province_Metro_Override_WOE.value_counts()

 0.165650    11
-0.389936    10
 0.253031     9
-0.161658     8
Name: Combo_Province_Metro_Override_WOE, dtype: int64

In [69]:
addon_impact.Combo_Province_Metro_Override_WOE.fillna(0, inplace=True)
addon_impact.Combo_Province_Metro_Override_WOE =addon_impact.Combo_Province_Metro_Override_WOE.round(8)

In [70]:
addon_impact.rename(columns={'impact':'addon_impact'},inplace=True)

In [71]:
#process to make sure the merge works;
rwa_raw_data.Combo_Province_Metro_Override_WOE=rwa_raw_data.Combo_Province_Metro_Override_WOE.round(6).fillna(0)
addon_impact.Combo_Province_Metro_Override_WOE = addon_impact.Combo_Province_Metro_Override_WOE.round(6).fillna(0)
addon_impact.Combo_LTV_Insured_Ind=addon_impact.Combo_LTV_Insured_Ind.fillna('')

In [72]:
rwa_raw_data_addon=rwa_raw_data.merge(addon_impact[['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE','addon_impact']], on=['Combo_LTV_Insured_Ind','Combo_Province_Metro_Override_WOE'],how='left')

In [73]:
rwa_raw_data_addon.addon_impact = rwa_raw_data_addon.addon_impact.fillna(1)

In [74]:
rwa_raw_data_addon['RWA_AddOn']=rwa_raw_data_addon['RWA_new']*rwa_raw_data_addon['addon_impact']

# Save Data

In [75]:

#with open(r'C:\Users\jasonz\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Concentra_RWA_202410.pkl', 'wb') as file: 
      
    # A new file will be created 
    #pickle.dump(rwa_raw_data_addon, file) 

# with open(r'C:\Users\francescav\OneDrive - Equitable Bank\EQB-Concentra Fit for Use - Fit_for_Use Development - Fit_for_Use Development\RWA\code\Outputs\Concentra_RWA_202412.pkl', 'wb') as file:   # 20250406 George commented out Francesca's code

with open( Output_dir +r'\concentra_rwa_addon.pkl', 'wb') as file: # 20250406 George added

    # A new file will be created 
    pickle.dump(rwa_raw_data_addon, file) 

In [76]:
#Save an excel copy

rwa_raw_data_addon.to_excel(Output_dir + r'\concentra_rwa_addon.xlsx')  #20250406 George added